# vLLM 완벽 가이드: 이론부터 실전까지

이 노트북에서는 vLLM을 이용한 효율적인 LLM 추론을 단계별로 학습합니다.

## 학습 목표
1. ✅ vLLM의 핵심 개념 이해 (PagedAttention, KV Cache)
2. ✅ vLLM 기본 사용법 실습
3. ✅ LoRA adapter 통합 방법
4. ✅ Transformers vs vLLM 성능 비교
5. ✅ 실전 응용 (Text-to-SQL)

## 사전 준비
```bash
pip install vllm==0.13.0 transformers torch peft
```

---
## Part 1: vLLM 기초 개념

### 1.1 왜 vLLM인가?

일반적인 Transformers 라이브러리의 문제점:
- **메모리 비효율**: KV Cache를 연속된 메모리에 저장 → 단편화 발생
- **느린 속도**: 배치 처리 시 비효율적
- **낮은 처리량**: 동시 요청 처리 능력 제한

vLLM의 해결책:
- **PagedAttention**: OS의 가상 메모리처럼 KV Cache를 블록 단위로 관리
- **연속 배칭**: 요청을 동적으로 배칭하여 처리량 극대화
- **최적화된 CUDA 커널**: GPU 활용도 향상

### 1.2 핵심 개념: PagedAttention

```
[기존 방식]
KV Cache: [████████░░░░░░░░]  ← 미리 큰 메모리 할당, 낭비 발생

[PagedAttention]
Block 0: [████]
Block 3: [████]  ← 필요할 때만 할당, 비연속적 저장 가능
Block 7: [██░░]
Block Table: [0, 3, 7]  ← 매핑 테이블로 관리
```

In [ ]:
# 필요한 라이브러리 import
import os
import time
import torch
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

print("✅ 라이브러리 로드 완료")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

---
## Part 2: vLLM 기본 사용법

### 2.1 모델 로딩

In [ ]:
# Step 1: vLLM 모델 초기화
model_name = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B"

print("🚀 vLLM 모델 로딩 중...")
start_time = time.time()

llm = LLM(
    model=model_name,
    tensor_parallel_size=1,       # 단일 GPU 사용
    gpu_memory_utilization=0.9,   # GPU 메모리 90% 사용
    trust_remote_code=True,       # 커스텀 모델 코드 신뢰
)

load_time = time.time() - start_time
print(f"✅ 모델 로딩 완료! ({load_time:.2f}초)")

# Tokenizer 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(f"📝 Tokenizer 준비 완료")

### 2.2 간단한 추론 실습

In [ ]:
# Step 2: Sampling 파라미터 설정
sampling_params = SamplingParams(
    max_tokens=128,        # 최대 생성 토큰 수
    temperature=0.7,       # 샘플링 온도 (높을수록 다양성 증가)
    top_p=0.9,            # Nucleus sampling
    top_k=50,             # Top-k sampling
)

print("⚙️  Sampling 파라미터:")
print(f"  - Max Tokens: {sampling_params.max_tokens}")
print(f"  - Temperature: {sampling_params.temperature}")
print(f"  - Top-p: {sampling_params.top_p}")

In [ ]:
# Step 3: 추론 실행
prompts = [
    "Explain quantum computing in simple terms:",
    "What is the capital of France?",
    "Write a haiku about programming:",
]

print("🤖 추론 시작...")
start_time = time.time()

outputs = llm.generate(prompts, sampling_params)

inference_time = time.time() - start_time

print(f"\n✅ 추론 완료! ({inference_time:.2f}초)")
print(f"📊 처리량: {len(prompts) / inference_time:.2f} prompts/sec\n")

# 결과 출력
for i, output in enumerate(outputs):
    generated_text = output.outputs[0].text
    token_count = len(output.outputs[0].token_ids)
    
    print(f"[프롬프트 {i+1}] {prompts[i]}")
    print(f"[생성 결과] {generated_text}")
    print(f"[토큰 수] {token_count}")
    print("-" * 60)

### 💡 실습 문제 1

**Q1**: `temperature`를 0.1, 0.7, 1.5로 바꿔가며 결과를 비교해보세요.
- 어떤 차이가 있나요?
- 어떤 상황에 어떤 값을 사용하면 좋을까요?

**Q2**: `max_tokens`를 32, 64, 256으로 변경하고 추론 시간을 측정해보세요.
- 토큰 수와 추론 시간의 관계는?

In [ ]:
# 실습 공간: 여기에 코드를 작성하세요



---
## Part 3: Chat 형식 프롬프트

실전에서는 시스템 메시지, 유저 메시지, 어시스턴트 메시지를 조합한 chat 형식을 많이 사용합니다.

In [ ]:
def format_chat_messages(messages):
    """
    Chat 메시지를 프롬프트로 변환
    
    Args:
        messages: [{'role': 'system/user/assistant', 'content': '...'}, ...]
    """
    if tokenizer.chat_template:
        # Tokenizer에 chat template이 있으면 사용
        return tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
    else:
        # 수동 포맷팅
        formatted = ""
        for msg in messages:
            role = msg['role'].capitalize()
            content = msg['content']
            formatted += f"{role}: {content}\n\n"
        
        if messages[-1]['role'] != 'assistant':
            formatted += "Assistant: "
        
        return formatted

print("✅ Chat 포맷팅 함수 준비 완료")

In [ ]:
# Text-to-SQL 예제
messages = [
    {
        'role': 'system',
        'content': 'You are a SQL expert. Convert natural language queries to SQL.'
    },
    {
        'role': 'user',
        'content': 'Find all users with age greater than 30'
    }
]

# 프롬프트 변환
prompt = format_chat_messages(messages)
print("📝 변환된 프롬프트:")
print(prompt)
print("-" * 60)

# 추론
sampling_params_sql = SamplingParams(
    max_tokens=64,
    temperature=0.1,  # SQL 생성은 낮은 temperature 사용
)

outputs = llm.generate([prompt], sampling_params_sql)
print("\n💡 생성된 SQL:")
print(outputs[0].outputs[0].text)

### 💡 실습 문제 2

다음 자연어 질의를 SQL로 변환하는 프롬프트를 작성하고 실행해보세요:

1. "Show top 10 products by sales"
2. "Count employees in each department"
3. "Find customers who haven't made a purchase in 6 months"

In [ ]:
# 실습 공간



---
## Part 4: LoRA Adapter 사용하기

### 4.1 LoRA란?

**LoRA (Low-Rank Adaptation)**:
- 모든 파라미터를 fine-tuning하는 대신, 작은 adapter만 학습
- 메모리 효율적이고 빠른 학습 가능
- 여러 task별 adapter를 쉽게 교체 가능

### 4.2 vLLM에서 LoRA 사용하는 두 가지 방법

#### 방법 1: Runtime LoRA (동적 적용)
- 추론 시점에 adapter를 동적으로 로드
- 여러 adapter를 빠르게 교체 가능

In [ ]:
# Runtime LoRA 예제 (adapter가 있는 경우)
lora_adapter_path = "./lora_adapter"  # 실제 adapter 경로로 변경

if os.path.exists(lora_adapter_path):
    from vllm.lora.request import LoRARequest
    
    # LoRA 지원 모델 로드
    llm_with_lora = LLM(
        model=model_name,
        enable_lora=True,
        max_lora_rank=64,
        tensor_parallel_size=1,
        gpu_memory_utilization=0.9,
        trust_remote_code=True,
    )
    
    # LoRA request 생성
    lora_request = LoRARequest("my_adapter", 1, lora_adapter_path)
    
    # 추론 (LoRA adapter 적용)
    outputs = llm_with_lora.generate(
        [prompt],
        sampling_params,
        lora_request=lora_request
    )
    
    print("✅ LoRA adapter 적용 완료")
else:
    print("⚠️ LoRA adapter 없음 - 건너뛰기")

#### 방법 2: Merged Model (사전 통합)
- LoRA weights를 base model에 미리 merge
- 더 빠른 추론 속도
- 배포에 유리

In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel

def merge_lora_to_base(base_model_name, lora_path, output_path):
    """
    LoRA adapter를 base model에 merge
    """
    print("🔄 LoRA merge 시작...")
    
    # 1. Base model 로드
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        trust_remote_code=True,
        torch_dtype="auto"
    )
    
    # 2. LoRA adapter 로드
    model_with_lora = PeftModel.from_pretrained(base_model, lora_path)
    
    # 3. Merge
    merged_model = model_with_lora.merge_and_unload()
    
    # 4. 저장
    os.makedirs(output_path, exist_ok=True)
    merged_model.save_pretrained(output_path, safe_serialization=True)
    
    # Tokenizer도 저장
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    tokenizer.save_pretrained(output_path)
    
    print(f"✅ Merge 완료: {output_path}")
    return output_path

# 예제 (adapter가 있는 경우)
if os.path.exists(lora_adapter_path):
    merged_path = merge_lora_to_base(
        base_model_name=model_name,
        lora_path=lora_adapter_path,
        output_path="./merged_model"
    )
    
    # Merged 모델로 추론
    llm_merged = LLM(
        model=merged_path,
        tensor_parallel_size=1,
        gpu_memory_utilization=0.9,
        trust_remote_code=True,
    )
    print("✅ Merged 모델로 추론 준비 완료")
else:
    print("⚠️ LoRA adapter 없음 - 건너뛰기")

---
## Part 5: 성능 벤치마크 - Transformers vs vLLM

vLLM이 얼마나 빠른지 직접 측정해봅시다!

In [ ]:
# 벤치마크 준비
test_prompts = [
    "You are a SQL expert. Convert this to SQL: Find all users",
    "You are a SQL expert. Convert this to SQL: Count employees",
    "You are a SQL expert. Convert this to SQL: Show top 10 sales",
    "You are a SQL expert. Convert this to SQL: Delete inactive accounts",
    "You are a SQL expert. Convert this to SQL: Update user emails",
]

max_tokens = 64

print(f"📊 벤치마크 설정")
print(f"  - 프롬프트 수: {len(test_prompts)}")
print(f"  - Max Tokens: {max_tokens}")

### 5.1 Transformers 벤치마크

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

print("🔵 Transformers 벤치마크 시작")
print("  모델 로딩 중...")

# 모델 로드
tf_tokenizer = AutoTokenizer.from_pretrained(model_name)
tf_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)
tf_model.eval()

# GPU 메모리 리셋
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()

# 추론
tf_start = time.time()
tf_total_tokens = 0
tf_first_token_latencies = []

for prompt in test_prompts:
    inputs = tf_tokenizer(prompt, return_tensors="pt").to(tf_model.device)
    
    # First token 측정
    ft_start = time.time()
    with torch.no_grad():
        outputs = tf_model(**inputs)
    first_token_time = time.time() - ft_start
    tf_first_token_latencies.append(first_token_time)
    
    # 전체 생성
    with torch.no_grad():
        generated = tf_model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=False,
        )
    tf_total_tokens += generated.shape[1] - inputs.input_ids.shape[1]

tf_time = time.time() - tf_start
tf_peak_memory = torch.cuda.max_memory_allocated() / 1024 / 1024 if torch.cuda.is_available() else 0
tf_avg_first_token = sum(tf_first_token_latencies) / len(tf_first_token_latencies)

print(f"\n✅ Transformers 완료")
print(f"  ⏱️  총 시간: {tf_time:.2f}s")
print(f"  ⚡ First Token: {tf_avg_first_token*1000:.2f}ms")
print(f"  🔥 Token/sec: {tf_total_tokens / tf_time:.2f}")
print(f"  💾 피크 메모리: {tf_peak_memory:.0f} MB")

# 메모리 정리
del tf_model
del tf_tokenizer
if torch.cuda.is_available():
    torch.cuda.empty_cache()

### 5.2 vLLM 벤치마크

In [ ]:
print("\n🟢 vLLM 벤치마크 시작")

# GPU 메모리 리셋
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()

# Sampling 파라미터
benchmark_params = SamplingParams(
    max_tokens=max_tokens,
    temperature=0.0,  # greedy decoding
)

# First token latency 측정
vllm_ft_start = time.time()
_ = llm.generate([test_prompts[0]], benchmark_params)
vllm_first_token = time.time() - vllm_ft_start

# 전체 추론
vllm_start = time.time()
vllm_outputs = llm.generate(test_prompts, benchmark_params)
vllm_time = time.time() - vllm_start

vllm_peak_memory = torch.cuda.max_memory_allocated() / 1024 / 1024 if torch.cuda.is_available() else 0
vllm_total_tokens = sum(len(out.outputs[0].token_ids) for out in vllm_outputs)

print(f"\n✅ vLLM 완료")
print(f"  ⏱️  총 시간: {vllm_time:.2f}s")
print(f"  ⚡ First Token: {vllm_first_token*1000:.2f}ms")
print(f"  🔥 Token/sec: {vllm_total_tokens / vllm_time:.2f}")
print(f"  💾 피크 메모리: {vllm_peak_memory:.0f} MB")

### 5.3 결과 비교

In [ ]:
print("\n" + "=" * 60)
print("📊 최종 비교 결과")
print("=" * 60)

# First Token Latency
print(f"\n⚡ First Token Latency:")
print(f"  Transformers: {tf_avg_first_token*1000:.2f}ms")
print(f"  vLLM:         {vllm_first_token*1000:.2f}ms")
ft_improvement = ((tf_avg_first_token - vllm_first_token) / tf_avg_first_token) * 100
print(f"  {'🚀 개선' if ft_improvement > 0 else '⚠️ 차이'}: {abs(ft_improvement):.1f}%")

# Token/sec
tf_tps = tf_total_tokens / tf_time
vllm_tps = vllm_total_tokens / vllm_time
print(f"\n🔥 Token/sec:")
print(f"  Transformers: {tf_tps:.2f} tokens/sec")
print(f"  vLLM:         {vllm_tps:.2f} tokens/sec")
print(f"  🚀 vLLM 향상: {vllm_tps / tf_tps:.2f}x")

# 총 시간
print(f"\n⏱️ 총 추론 시간:")
print(f"  Transformers: {tf_time:.2f}s")
print(f"  vLLM:         {vllm_time:.2f}s")
print(f"  🚀 속도 향상: {tf_time / vllm_time:.2f}x")

# 메모리
print(f"\n💾 피크 GPU 메모리:")
print(f"  Transformers: {tf_peak_memory:.0f} MB")
print(f"  vLLM:         {vllm_peak_memory:.0f} MB")
memory_diff = tf_peak_memory - vllm_peak_memory
if memory_diff > 0:
    print(f"  💡 메모리 절감: {memory_diff:.0f} MB ({memory_diff/tf_peak_memory*100:.1f}%)")
else:
    print(f"  📈 메모리 사용: {abs(memory_diff):.0f} MB 더 사용")

print("\n" + "=" * 60)

### 💡 분석 포인트

벤치마크 결과를 보면:

1. **First Token Latency**: vLLM이 더 빠른 이유는?
   - 최적화된 CUDA 커널
   - 효율적인 메모리 관리

2. **Token/sec**: vLLM이 압도적인 이유는?
   - PagedAttention의 효율성
   - 연속 배칭 (Continuous Batching)
   - KV Cache 재사용

3. **메모리**: 왜 절감되는가?
   - 동적 메모리 할당
   - 메모리 단편화 방지
   - 블록 단위 관리

4. **배치 처리**: 프롬프트가 많을수록 vLLM의 이점이 커집니다!

---
## Part 6: 종합 실습

### 최종 프로젝트: Text-to-SQL 시스템 구축

배운 내용을 종합하여 실전 Text-to-SQL 시스템을 만들어봅시다.

In [ ]:
class TextToSQLSystem:
    def __init__(self, model_name):
        self.llm = LLM(
            model=model_name,
            tensor_parallel_size=1,
            gpu_memory_utilization=0.9,
            trust_remote_code=True,
        )
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        self.system_prompt = """
You are an expert SQL query generator. Convert natural language questions to SQL queries.
Generate only the SQL query without explanations.
"""
    
    def generate_sql(self, question: str, schema_info: str = "") -> str:
        """자연어 질의를 SQL로 변환"""
        
        user_content = f"""
Database Schema:
{schema_info if schema_info else 'Standard database tables'}

Question: {question}

SQL Query:
"""
        
        messages = [
            {'role': 'system', 'content': self.system_prompt},
            {'role': 'user', 'content': user_content}
        ]
        
        prompt = format_chat_messages(messages)
        
        sampling_params = SamplingParams(
            max_tokens=128,
            temperature=0.1,
            stop=[";", "\n\n"]
        )
        
        outputs = self.llm.generate([prompt], sampling_params)
        sql_query = outputs[0].outputs[0].text.strip()
        
        return sql_query

# 시스템 초기화
sql_system = TextToSQLSystem(model_name)
print("✅ Text-to-SQL 시스템 준비 완료")

In [ ]:
# 테스트
schema = """
Tables:
- users (id, name, email, age, department_id)
- departments (id, name, budget)
- orders (id, user_id, product_id, amount, order_date)
"""

test_questions = [
    "Find all users older than 30",
    "Count employees in each department",
    "Show top 5 users by order amount",
    "List departments with budget over 100000",
]

print("🧪 Text-to-SQL 테스트\n")

for question in test_questions:
    print(f"❓ Question: {question}")
    sql = sql_system.generate_sql(question, schema)
    print(f"💡 Generated SQL: {sql}")
    print("-" * 60)

### 💡 최종 실습 문제

**과제**: `TextToSQLSystem`을 개선해보세요!

1. **Few-shot Learning 추가**:
   - 시스템 프롬프트에 예제 query-SQL 페어 추가
   - 성능이 개선되나요?

2. **에러 처리**:
   - 잘못된 SQL이 생성되면 재시도하는 로직 추가
   - SQL 유효성 검증 추가

3. **배치 처리**:
   - 여러 질의를 한 번에 처리하는 기능 추가
   - 성능 차이 측정

4. **성능 최적화**:
   - 다양한 sampling parameters 실험
   - 최적의 설정 찾기

In [ ]:
# 최종 실습 공간



---
## 정리 및 요약

### ✅ 학습한 내용

1. **vLLM 핵심 개념**
   - PagedAttention: 블록 단위 메모리 관리
   - KV Cache: 효율적인 캐싱 메커니즘
   - 연속 배칭: 동적 배치 처리

2. **실전 사용법**
   - 모델 로딩 및 초기화
   - Sampling parameters 설정
   - Chat 형식 프롬프트 처리

3. **LoRA 통합**
   - Runtime LoRA: 동적 적용
   - Merged Model: 사전 통합

4. **성능 최적화**
   - Transformers 대비 2-3배 빠른 속도
   - 메모리 효율성 개선
   - 높은 처리량

### 🚀 다음 단계

1. **프로덕션 배포**
   - FastAPI와 통합
   - 멀티 GPU 설정
   - 모니터링 및 로깅

2. **고급 기능**
   - Multi-LoRA: 여러 adapter 동시 사용
   - Prefix Caching: 공통 프롬프트 캐싱
   - Speculative Decoding: 추론 가속화

3. **실전 응용**
   - 챗봇 시스템
   - 코드 생성 도구
   - 문서 요약 서비스

### 📚 참고 자료

- [vLLM 공식 문서](https://docs.vllm.ai/)
- [PagedAttention 논문](https://arxiv.org/abs/2309.06180)
- [LoRA 논문](https://arxiv.org/abs/2106.09685)

---

**🎉 수고하셨습니다! 이제 여러분은 vLLM 마스터입니다!**